In [1]:
import pandas as pd # для обработки и анализа данных
import numpy as np # для математических вычислений
from tqdm.auto import tqdm # для отслеживания прогресса длительных операций
import matplotlib.pyplot as plt # для визуализации
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import re
from string import punctuation
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.models import Word2Vec
import gensim.downloader as api
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
from sklearn.manifold import TSNE
from typing import List

In [4]:
re.sub(pattern=r'<.*>', repl='replace_this', string='<a> b <c>')
re.sub(pattern=r'<.*?>', repl='replace_this', string='<a> b <c>')

'replace_this b replace_this'

In [12]:
s = '111'
print(s)

111


In [ ]:
re.sub(pattern=r'\{\{.*?\}\}', repl='', string='{{111}}12345{{000}}trtrt')

'12345trtrt'

In [2]:
def remove_private_data(texts: List[str]) -> List[str]:
    return [re.sub(pattern=r'\{\{.*?\}\}', repl='', string=x) for x in texts]

In [5]:
texts : List[str] = ['{{111}}111{222}}aaa', '222', '333']
remove_private_data(texts)

['111{222}}aaa', '222', '333']